In [1]:
%cd /home/sasedov/StyleDomain/SimilarDomains

/home/sasedov/StyleDomain/SimilarDomains


/home/sasedov/.conda/envs/StyleDomain-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import PIL
import random

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor
from omegaconf import OmegaConf
from core.utils.common import load_clip, mixing_noise
from core.utils.example_utils import (
    Inferencer, to_im, vstack_with_lines, hstack_with_lines, insert_image,
    project_e4e, project_restyle_psp, project_fse_without_image_generation, read_img
)
from core.utils.image_utils import construct_paper_image_grid
from core.utils.reading_weights import read_weights
from core.uda_models import OffsetsTunningGenerator

from pathlib import Path
from collections import defaultdict

from examples.draw_util import weights, set_seed, morph_g_ema, IdentityEditor, StyleEditor

from core.utils.example_utils import read_fse_config
scale, scale_mode, idx_k, n_styles, enc_residual, enc_residual_coeff, resnet_layers, stride = read_fse_config('001')
print(scale, scale_mode, idx_k, n_styles, enc_residual, enc_residual_coeff, resnet_layers, stride)

/home/sasedov/.conda/envs/StyleDomain-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2 bilinear 5 18 False False [4, 5, 6] (2, 2)


In [3]:
def stack_rows(rows, skip_horiz=10, skip_vertical=15):
    final_image = [
        hstack_with_lines(row_stack, skip_horiz) for row_stack in rows
    ]

    final_image = vstack_with_lines(final_image, skip_vertical)
    return PIL.Image.fromarray(final_image)


style_to_editor = {
    d: StyleEditor(read_weights(weights[d])) for d in weights if d not in ['horse', 'car', 'ffhq', 'cat', 'church', 'to_metfaces', 'to_afhqcat', 'to_afhqdog', 'to_mega']
}

style_to_editor['original'] = IdentityEditor()
device = 'cuda:0'

In [8]:
ws_e4e = torch.load(f"ws_e4e_tensor.pt", map_location='cuda:0')
ws_fse = torch.load(f"ws_fse_tensor.pt", map_location='cuda:0')
fse_features_tensor = torch.load("fse_features_tesnor.pt", map_location=device)
orig_features_tensor_src = torch.load("orig_features_tensor_src.pt", map_location=device)
orig_features_tensor_trg = torch.load("orig_features_tensor_trg.pt", map_location=device)

filenames = []
with open('latent_corr_exp_filenames.txt', 'r') as f:
    for line in f.readlines():
        filenames.append(line.strip())

In [5]:
from contextlib import redirect_stdout
from visualize import stack_to_grid_with_names
from PIL import ImageFont
import io
import sys

def test_inversion_on_different_encoders(
        gan_domain, s_domain, ims,
        w_enc_list, features_list, generator_shift_list,
        image_size=256, verbose=False,
        row_names=None, column_names=None
    ):
    """
        ims: list of initial images
        w_enc_list: list of lists latents for each encoder
        features_list: list of lists of encoder feature maps for each image and encoder, should be None by default
        generator_shift_list: list of lists of flags, whether to shift with generator feature maps
    """
    resize = Resize(image_size)
    transform = transforms.Compose(
        [
            transforms.Resize((1024, 1024)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ]
    )
    
    def prepare_im(im, use_transform=False, unsqueeze=False):
        if use_transform: 
            im = transform(im)
        if unsqueeze:
            im = im.unsqueeze(0)
        return np.array(to_im(resize(im), padding=0))
    
    row_stack = []
    row = []
    
    assert len(w_enc_list) == len(features_list) and len(w_enc_list) == len(generator_shift_list)
    images_per_row = len(w_enc_list[0]) + 1

    if not verbose:
        f_stdout = io.StringIO()

    for i, im in enumerate(ims):
        ckpt = read_weights(weights[s_domain])
        ckpt_ffhq = {'sg2_params': ckpt['sg2_params']}
        ckpt_ffhq['sg2_params']['checkpoint_path'] = weights[gan_domain]

        if not verbose:
            with redirect_stdout(f_stdout):
                model = Inferencer(ckpt, device)
        
        row.append(prepare_im(im, use_transform=True, unsqueeze=True))
        
        w_enc_im_list = w_enc_list[i]
        features_im_list = features_list[i]
        generator_shift_im_list = generator_shift_list[i]
        
        for j, (w_enc, features, generator_shift) in enumerate(zip(w_enc_im_list, features_im_list, generator_shift_im_list)):
            kwargs = {
                "latents": [w_enc],
                "input_is_latent": True,
                "features_in": features,
                "shift_with_generator_feature_map": generator_shift
            }
            if not verbose:
                with redirect_stdout(f_stdout):
                    src, trg = model(**kwargs)
            else:
                src, trg = model(**kwargs)
                
            # add original image and its inversion to row_stack
            row.append(prepare_im(src))

        row_stack.append(row)
        row = []

    return stack_to_grid_with_names(
        imgs_list=row_stack, H=image_size, W=image_size,
        row_names=row_names, column_names=column_names,
        font=ImageFont.truetype("/home/sasedov/Times.ttf", 25 * image_size // 256),
    )

In [6]:
from contextlib import redirect_stdout
import io


def inverse_images(indices=None, n_images=16, use_random_indices=True, dataset_path='/home/sasedov/StyleDomain/faces_dataset_small/'):
    if use_random_indices:
        indices = np.random.randint(0, len(filenames), n_images)
    else:
        n_images = len(indices)

    ims = []
    ws_ims = []
    fse_features_ims = []
    generator_shift_ims = []

    f_stdout = io.StringIO()

    for i in indices:
        im = read_img(dataset_path + filenames[i], align_input=False)
        ims.append(im)

        with redirect_stdout(f_stdout):
            w_e4e = project_e4e(im, 'pretrained/e4e_ffhq_encode.pt')[1]
            empty_img, w_fse, fse_features = project_fse_without_image_generation(
                im,
                model_path='pretrained/143_enc.pth',
                fse_config_name='001',
                arcface_model_path='pretrained/backbone.pth',
                stylegan_model_path='pretrained/StyleGAN2/stylegan2-ffhq-config-f.pt'
            )
        ws_ims.append([w_e4e, w_fse, w_fse, w_fse])
        fse_features_ims.append([None, None, fse_features, fse_features])
        generator_shift_ims.append([False, False, False, True])

    print('List of used encoders:')
    print('1) E4E latents')
    print('2) FSE with latents only')
    print('3) FSE with latents and feature map')
    print('4) FSE with latents, feature map and generator shift')
    
    return ims, ws_ims, fse_features_ims, generator_shift_ims, indices

In [ ]:
ims, ws_ims, fse_features_ims, generator_shift_ims, random_indices = inverse_images(n_images=24)
result = test_inversion_on_different_encoders(
    gan_domain='ffhq', s_domain='murasaki_nora_asuya', ims=ims,
    w_enc_list=ws_ims,
    features_list=fse_features_ims, 
    generator_shift_list=generator_shift_ims,
    image_size=256, verbose=False
)
result

In [ ]:
# filenames[974], filenames[2371], filenames[129], filenames[504], filenames[40], filenames[316], filenames[418], 
worthful_indices = [974, 2371, 129, 504, 50, 144, 1883, 1597, 316, 418, 2898, 1441, 749, 713, 1505, 360, 2455, 132, 967, 1301, 546, 887, 2402, 2252]
worthful_indices += [2372, 2747, 2861, 1695, 114, 2515, 798, 1677, 2089, 728, 2404, 1536, 1733, 1215, 1868, 3003, 2817, 2686]

ims, ws_ims, fse_features_ims, generator_shift_ims, _ = inverse_images(
    indices=worthful_indices,
    use_random_indices=False
)

result = test_inversion_on_different_encoders(
    gan_domain='ffhq', s_domain='jojo', ims=ims,
    w_enc_list=ws_ims,
    features_list=fse_features_ims, 
    generator_shift_list=generator_shift_ims,
    image_size=1024, verbose=False,
    row_names=[filenames[x] for x in worthful_indices],
    column_names=["orig", "e4e latents", "fse latents", "fse features", "fse generator shift"]
)
result.save('inversion_comparison_3.pdf', save_all=True, append_images=[])
result